<a id="top"></a>
# Localization-delocalization transition in the physical Laplacian spectrum

In [1]:
import igraph as ig
import numpy as np
from scipy.optimize import root_scalar
from scipy.sparse import coo_array
from scipy.sparse.linalg import eigsh

import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

In [2]:
def inverse_participation_ratio(vec):
    """Calculate the inverse participation ratio of a vector.
    
    Parameters
    ----------
    vec : np.array
        The vector for which to calculate the inverse participation ratio.
    
    Returns
    -------
    ipr : float
        The inverse participation ratio of the vector.
    """
    return np.sum(vec**4.)/(np.sum(vec**2.)**2.)

In [3]:
def physical_eigen_values(g, vols, which='leading'):
    """Calculate the eigenvalue of the physical Laplacian.

    Parameters
    ----------
    g : igraph.Graph
        The graph for which to calculate the eigenvalue.
    
    vols : np.array
        The volume of each vertex in the graph.
    
    which : str
        Which eigenvalue to calculate. Options are 'leading' and 'fiedler'.
    
    Returns
    -------
    eigs[0] : float
        The eigenvalue of the physical Laplacian.
    """
    
    row=[]
    col=[]
    val=[]
    for e in g.es:
        row.append(e.source)
        col.append(e.target)
        val.append(-1)

        row.append(e.target)
        col.append(e.source)
        val.append(-1)

    for vid, k in enumerate(g.degree()):
        row.append(vid)
        col.append(vid)
        val.append(k)
    
    Lap = coo_array((val, (row, col)), shape=(g.vcount(), g.vcount())).tocsc()
    
    row=[]
    col=[]
    val=[]
    for v in g.vs:
        row.append(v.index)
        col.append(v.index)
        val.append(vols[v.index])
    val = np.array(val)
    val = val**-.5

    vs = coo_array((val, (row, col)), shape=(g.vcount(), g.vcount())).tocsc()

    physLap = vs.dot(Lap.dot(vs))
    
    if which == 'leading':
        eigs, vecs = eigsh(physLap, k=1, which='LM', return_eigenvectors=True)
        return eigs[0], vecs[:,0]
    elif which == 'fiedler':
        eigs, vecs = eigsh(physLap, k=2, which='SM', return_eigenvectors=True)
        return eigs[1], vecs[:,1]

## Regular random graphs with a hub

In [ ]:
# def g(z,c):
#     return ((c - 2) * z - np.sign(z) * c * (z**2 - 4 * (c - 1))**.5) / (2 * (c**2 - z**2))

# def Dg(z,c):
#     return (
#         (c - 2 - np.sign(z) * c * z / (z**2 - 4 * (c - 1))**.5) / (2 * (c**2 - z**2))
#         + z * ((c - 2) * z - np.sign(z) * c * (z**2 - 4 * (c - 1))**.5) / (c**2 - z**2)**2
#     )

In [ ]:
n_nodes = 1000
degree = 4

## Erdos-Renyi random graphs with a hub

In [ ]:
n_nodes = 1000
avg_degree = 4


## Configuration model

In [ ]:
n_nodes = 1000
avg_degree = 4
n_edges = n_nodes * avg_degree // 2
gamma = 2.5

g = ig.Graph.Static_Power_Law(
    n=n_nodes, m=n_edges, exponent_out=gamma, direct=False
)
deg = np.array(g.degree())

## Barabasi-Albert model

In [4]:
n_nodes = 1000
m = 2

g = ig.Graph.Barabasi(n=n_nodes, m=m)
deg = np.array(g.degree())

In [5]:
n_steps = 21
alpha_min = 0.
alpha_max = 2.

alphas = np.linspace(alpha_min, alpha_max, n_steps)

In [ ]:
for alpha in alphas:
    vols = deg**alpha
    eigs, vec = physical_eigen_values(g, vols, which='leading')
